In [22]:
import uvicorn
from threading import Thread
from fastapi import FastAPI, File, UploadFile
from fastapi.middleware.cors import CORSMiddleware
import numpy as np
import tensorflow as tf
from tensorflow import keras
from PIL import Image
from io import BytesIO

# Initialize FastAPI app
app = FastAPI()

# CORS Middleware for frontend integration
origins = ["http://localhost", "http://localhost:3000"]
app.add_middleware(
    CORSMiddleware,
    allow_origins=origins,
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# Load the trained model (ensure you have converted to .h5 format)
MODEL_PATH = r"C:\Users\Dell\deep learning\potato_disease_classification\model.h5"
MODEL = keras.models.load_model(MODEL_PATH)  # Load the model correctly

# Define class names
CLASS_NAMES = ["Early Blight", "Late Blight", "Healthy"]

# Function to process uploaded image
def read_file_as_image(data) -> np.ndarray:
    image = Image.open(BytesIO(data)).convert("RGB")  # Ensure image is in RGB mode
    image = image.resize((256, 256))  # Resize to match model input size
    image = np.array(image) / 255.0  # Normalize pixel values
    return np.expand_dims(image, axis=0)  # Add batch dimension

# Prediction endpoint
@app.post('/predict')
async def predict(file: UploadFile = File(...)):
    try:
        image = read_file_as_image(await file.read())  # Read and preprocess image
        predictions = MODEL.predict(image)  # Make predictions
        
        predicted_class = CLASS_NAMES[np.argmax(predictions[0])]
        confidence = float(np.max(predictions[0]))

        return {
            'class': predicted_class,
            'confidence': confidence
        }
    
    except Exception as e:
        return {"error": str(e)}

# Function to run FastAPI in a background thread
def run_fastapi():
    uvicorn.run(app, host="localhost", port=8000)

# Start FastAPI in a separate thread
Thread(target=run_fastapi, daemon=True).start()


ERROR:asyncio:Task exception was never retrieved
future: <Task finished name='Task-14' coro=<Server.serve() done, defined at C:\Users\Dell\anaconda3\Lib\site-packages\uvicorn\server.py:68> exception=SystemExit(1)>
Traceback (most recent call last):
  File "C:\Users\Dell\anaconda3\Lib\site-packages\uvicorn\server.py", line 163, in startup
    server = await loop.create_server(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Dell\anaconda3\Lib\asyncio\base_events.py", line 1571, in create_server
    raise OSError(err.errno, msg) from None
OSError: [Errno 10048] error while attempting to bind on address ('::1', 8000, 0, 0): [winerror 10048] only one usage of each socket address (protocol/network address/port) is normally permitted

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Dell\anaconda3\Lib\threading.py", line 1075, in _bootstrap_inner
    self.run()
  File "C:\Users\Dell\anaconda3\Lib\threading.py", l

In [ ]:
!streamlit run app.py


INFO:     Started server process [7348]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
ERROR:    [Errno 10048] error while attempting to bind on address ('::1', 8000, 0, 0): [winerror 10048] only one usage of each socket address (protocol/network address/port) is normally permitted
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
ERROR:asyncio:Task exception was never retrieved
future: <Task finished name='Task-17' coro=<Server.serve() done, defined at C:\Users\Dell\anaconda3\Lib\site-packages\uvicorn\server.py:68> exception=SystemExit(1)>
Traceback (most recent call last):
  File "C:\Users\Dell\anaconda3\Lib\site-packages\uvicorn\server.py", line 163, in startup
    server = await loop.create_server(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Dell\anaconda3\Lib\asyncio\base_events.py", line 1571, in create_server
    raise OSError(err.errno, msg) from None
OSError: [Errno 10048] error while attempting

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
INFO:     ::1:56733 - "POST /predict HTTP/1.1" 200 OK
